In [6]:
# automatically reload dependant notebooks
%load_ext autoreload
%autoreload 2
import import_ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Single-Source Shortest Paths

Given a directed, weighted graph $G = (V, E)$, Single-Source Shortest Path (SSP) algorithm discovers the shortest path $\delta(u, v)$ with the minimum total weight if the path $u \leadsto v$ exists, or $\infty$ if the path does not exist. See CLRS 4ed Chapter 22 *Single-Source Shortest Paths* p.604. SSP is used by the car's GPS navigation unit to select the shortest path, out of many possible ones, between the current location and the chosen destination.

Since the graphs used in this notebook employ weighted edges, we shall define `SSPGraph` to be `MSTGraph` as defined in [`mst.ipynb`](./mst.ipynb).

In [7]:
from graph import *
from mst import *
from util import *

SSPGraph = MSTGraph

## Bellman-Ford SSP algorithm

Bellman-Ford SSP algorithm relaxes each edge iteratively until all simple paths from the source vertex $s$ to every other vertex becomes the shortest path $\delta(u, v)$. The algorithm works with negative weights. It detects if a negative-weight cycle is reachable from the source vertex `s`, and if so, it returns a `None`, indicating that there exists no shortest path. See §22.1 *The Bellman-Ford algorithm* p.612. The runtime of Bellman-Ford algorithm is $O(VE)$. See p.613.

We also define below the initialisation sequence `init()` as given on p.609 and the `relax()` function as given on p.610. Moreover, we factored out SSP extraction into `getSSP()`, so that we may reuse it elsewhere.

In [8]:
def init(g: SSPGraph, s: Vert) -> None:
  # see p.609
  for u in g.getVV():
    u.par = None
    u.dis = Infinity
  s.dis = 0

def relax(e: WgtEdge) -> bool:
  # return True if v.dis is decreased, False otherwise; see pp.610,620
  u = e.u
  v = e.v
  d = u.dis + e.wgt
  if v.dis > d:
    v.par = u
    v.dis = d
    return True
  return False

def shortestPathWeight(g: SSPGraph, s: Vert, v: Vert) -> float:
  # see p.604
  return [g.getE(makeETag(u.par, u)).wgt for u in path(s, v)]

def sspBellmanFord(g: SSPGraph, s: Vert) -> Option[Tree]:
  # initialize
  init(g, s)
  # relax edges
  n = g.numVV()
  for i in range(1, n):  # iteration i ∈ [1, n)
    for e in g.getEE(): relax(e)
  # check for negative-weight cycle
  for e in g.getEE():
    u = e.u
    v = e.v
    if v.dis > u.dis + e.wgt: return None  # found negative-weight cycle reachable from vertex s
  return getSSP(g, s)  # extract SSP p from graph g

def getSSP(g: SSPGraph, s: Union[Vert, PriVert]) -> Tree:
  p = Tree(f"{g.tag}¶")
  p.insV(s)
  for u in g.getVV():
    if u != s:
      p.insV(u)
    if not u.isRoot():
      etag = makeETag(u.par, u)
      if g.hasE(etag):
        e = g.getE(etag)
        p.insE(e)
  return p

## Bellman-Ford DAWG

Now, we implement the `sspBellmanFordDAWG()`, which first applies topological sort to the vertices to achieve a better runtime when computing SSP on a directed, acyclic, weighted graph (DAWG). See §22.2 *Single-source shortest paths in directed acyclic graphs* p.616. This algorithm can be used to find critical paths in [Programme Evaluation and Review Technique](https://en.wikipedia.org/wiki/Program_evaluation_and_review_technique) (PERT) charts, the darlings of managers the world over. The word "programme" in PERT refers to government contract programme, not computer programme. The runtime of Bellman-Ford algorithm on a DAWG is faster and tighter: $\Theta(V + E)$. See p.617.

Note that CLRS calls this type of graph "weighted, directed, acyclic graph", but I call it "directed, acyclic, weighted graph", simply because I favour the acronym "DAWG".

In [9]:
def sspBellmanFordDAWG(g: SSPGraph, s: Vert) -> Option[Tree]:
  vv = tsort(g)
  # initialize
  init(g, s)
  # relax edges
  for u in vv:  # for each topologically sorted vertex
    for v in g.adj(u): relax(g.getE(makeETag(u, v)))
  return getSSP(g, s)

## Dijkstra's SSP algorithm

Dijkstra's SSP algorithm follows the structure of BFS. But where as BFS assumes unitary weight edges, Dijkstra's algorithm works with non-negative edge weights. See 22.3 *Dijkstra’s algorithm* p.620. The runtime of Dijkstra's algorithm is $O(V^2)$. See p.623.


Dijkstra's algorithm uses `PriorityQueue`. When we implemented `PrimMSTGraph` in [`mst.ipynb`](./mst.ipynb), we defined `PriVertex` which can be stored in a priority queue. We shall reuse it here. But since Dijkstra's algorithm only updates the `dis` attribute, we manually copy the updated `dis` value to the `pri` attribute, so as to make the priority queue work correctly.

In [10]:
from functools import reduce
from queue import PriorityQueue

DijkstraSSPGraph = PrimMSTGraph  # uses PriVertex

def sspDijkstra(g: SSPGraph, s: PriVert) -> Tree:
  assert(reduce(lambda acc, e: acc and e.wgt >= 0, g.getEE(), True))
  # initialize
  init(g, s)
  s.pri = s.dis
  b: VSet = {}  # vertex set of SSP
  q = PriorityQueue()
  for u in g.getVV(): q.put(u)
  # discover SSP in graph g
  while not q.empty():
    u = q.get()
    b[u.tag] = u
    for v in g.adj(u):
      if relax(g.getE(makeETag(u, v))):
        v.pri = v.dis
        q.queue.sort()  # rearrange q to account for decreased v.dis
  return getSSP(g, s)

Note that Dijkstra's SSP algorithm works only with non-negative edge weights. So, we assert this fact before we proceed with the computation of the SSP.

# Conclusion

In this notebook, we implemented Bellman-Ford algorithm and Dijkstra's algorithm for computing single-source shortest paths. Dijkstra's algorithm is the more efficient of the two. Tests for these SSP algorithms are implemented in [`ssptest.ipynb`](./ssptest.ipynb).